In [1]:
import pandas as pd
import json

In [2]:
with open('/tmp/pycharm_project_366/config.json') as f:
    config = json.load(f)

In [3]:
def get_people_with_disease(df, all_diseased_column, disease_code_pattern):
    people_with_disease_df = df[all_diseased_column.str.contains(disease_code_pattern)]

    return people_with_disease_df

In [6]:
test_group_df = pd.read_csv('/tmp/pycharm_project_366/test_data.csv', low_memory=False)
test_group_df

In [8]:
diabetes_pattern = r'E11'
pancreatic_cancer_pattern = r'C25'
all_diseased_column = test_group_df[[f'41270-0.{i}' for i in range(100)]].agg(', '.join, axis=1)
people_with_diabetes_in_test_df = get_people_with_disease(test_group_df, all_diseased_column, diabetes_pattern)
people_with_pancreatic_cancer_in_test_df = get_people_with_disease(test_group_df, all_diseased_column, pancreatic_cancer_pattern)
print(f'people with diabetes: {people_with_diabetes_in_test_df.shape[0]}, people with cancer: {people_with_pancreatic_cancer_in_test_df.shape[0]}')

In [9]:
train_group_df = pd.read_csv('/tmp/pycharm_project_366/train_data.csv', low_memory=False)
train_group_df

In [10]:
diabetes_pattern = r'E11'
pancreatic_cancer_pattern = r'C25'
all_diseased_column = train_group_df[[f'41270-0.{i}' for i in range(100)]].agg(', '.join, axis=1)
people_with_diabetes_in_train_df = get_people_with_disease(train_group_df, all_diseased_column, diabetes_pattern)
people_with_pancreatic_cancer_in_train_df = get_people_with_disease(train_group_df, all_diseased_column, pancreatic_cancer_pattern)
print(f'people with diabetes: {people_with_diabetes_in_train_df.shape[0]}, people with cancer: {people_with_pancreatic_cancer_in_train_df.shape[0]}')

In [5]:
df_test.iloc[:,270]

In [10]:
df_train = pd.read_csv('/tmp/pycharm_project_366/train_data.csv')
df_train

In [9]:
set(df_test.columns) - set(config['features_code_list'])

In [10]:
'41270-0.99.1' in config['features_code_list']

In [11]:
features = pd.read_csv('/home/ofeksh2@mta.ac.il/biobank/fields672220.ukb')

In [21]:
'21028-0.0.1' in config['features_code_list']